<a href="https://colab.research.google.com/github/aaronbbarclay/mine/blob/master/mask_rcnn_test_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%mkdir /root/data
%cd /root/data

/root/data


In [0]:
%cd /root/data
!git clone https://github.com/matterport/Mask_RCNN

/root/data
Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 119.40 MiB | 32.65 MiB/s, done.
Resolving deltas: 100% (568/568), done.


In [9]:
import sys
sys.path.append("/root/data/Mask_RCNN")
%cd /root/

import mrcnn

/root


In [0]:
%mkdir -p /content/workspace
%cd /content/workspace

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /content/local_drive/


/content/workspace
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
%cd /

/


In [3]:
%cd /content/drive/My\ Drive/MachineLearning/trainingSets/srcVids
%ls

/content/drive/My Drive/MachineLearning/trainingSets/srcVids
'Cooper Station Scene.mp4'
 dune_trailer_1984_360p.mp4
 family_reunion_and_farewell_ezra_and_thrawn_star_wars_rebels_disney_xd.mp4
 IMG_0054.MOV
 IMG_0357.MOV
 IMG_1638.MOV
 interstellar1.mp4
'Interstellar - Atmospheric Entry Scene 1080p HD.mp4'
 jim_gaffigan_obsessed_weddings.mp4


In [6]:
%cd /content/local_drive/srcVids/
%ls -lh

/content/local_drive/srcVids
total 156M
-rw------- 1 root root  21M Jun  5 11:13 'Cooper Station Scene.mp4'
-rw------- 1 root root 7.7M Jun  5 11:13  dune_trailer_1984_360p.mp4
-rw------- 1 root root  26M Jun  5 11:13  family_reunion_and_farewell_ezra_and_thrawn_star_wars_rebels_disney_xd.mp4
-rw------- 1 root root  44M Jun  5 11:13  IMG_0054.MOV
-rw------- 1 root root  14M Jun  5 11:13  IMG_0357.MOV
-rw------- 1 root root 8.6M Jun  5 11:13  IMG_1638.MOV
-rw------- 1 root root 6.4M Jun  5 11:13  interstellar1.mp4
-rw------- 1 root root  22M Jun  5 11:13 'Interstellar - Atmospheric Entry Scene 1080p HD.mp4'
-rw------- 1 root root 7.3M Jun  5 11:13  jim_gaffigan_obsessed_weddings.mp4


In [19]:
%cd /content
!rm -rf /content/local_drive
!mkdir -p /content/local_drive/
!cp -r /content/drive/My\ Drive/MachineLearning/trainingSets/srcVids /content/local_drive/srcVids


/content


In [20]:
%cd /content/local_drive/
%ls
%cd /content/local_drive/srcVids/
%ls

/content/local_drive
srcVids/
/content/local_drive/srcVids
'Cooper Station Scene.mp4'
 dune_trailer_1984_360p.mp4
 family_reunion_and_farewell_ezra_and_thrawn_star_wars_rebels_disney_xd.mp4
 IMG_0054.MOV
 IMG_0357.MOV
 IMG_1638.MOV
 interstellar1.mp4
'Interstellar - Atmospheric Entry Scene 1080p HD.mp4'
 jim_gaffigan_obsessed_weddings.mp4


In [10]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/root/data/Mask_RCNN")

from mrcnn import utils
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")


Using TensorFlow backend.


In [11]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [12]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [0]:
# Forked form the visualize module as I want to save out the results of this function (I need it to return the reulst.)


import os
import sys
import random
import itertools
import colorsys

import numpy as np
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import IPython.display

def display_and_return_instances(image, boxes, masks, class_ids, class_names,
                        scores=None, title="",
                        figsize=(16, 16), ax=None,
                        show_mask=True, show_bbox=True,
                        colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    #if not ax:
    #    _, ax = plt.subplots(1, figsize=figsize)
    #    auto_show = True

    # Generate random colors
    colors = colors or visualize.random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    #ax.set_ylim(height + 10, -10)
    #ax.set_xlim(-10, width + 10)
    #ax.axis('off')
    #ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            #ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        #ax.text(x1, y1 + 8, caption,
        #        color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = visualize.apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            #ax.add_patch(p)
    #ax.imshow(masked_image.astype(np.uint8))
    #if auto_show:
    #    plt.show()
        
    return masked_image.astype(np.uint8)

In [0]:
%cd /content/local_drive/MachineLearning/trainingSets/someVids/

%cd IMG_0357/frames


!ffmpeg -r 25 -f image2 -i inference.%04d.jpg -vcodec libx264 -crf 10 -pix_fmt yuv420p IMG_0357.mp4

In [53]:
import os.path
from PIL import Image
import numpy
import re

# Load a random image from the images folder
VID_DIR = "/content/local_drive/srcVids"
FRAMES_DIR = "/content/local_drive/FRAMES"
OUT_DIR = "/content/local_drive/OUT"

    
escaped_vid = VID_DIR.replace(" ", "\ ")
print(escaped_vid)

!mkdir -p $OUT_DIR

srcVids = os.listdir(VID_DIR)

for vid in srcVids[:2]:

    print("#"*80)
    print("  ----- Decoding ----")
    print(vid)
    print("#"*80)
    
    image_path = os.path.join(VID_DIR, vid)
    #print(image_path)
    image_base, image_extension = vid.split(".")

    if image_extension not in ["mov", "mp4", "MOV"]:
        continue
        
    #print(image_base, image_extension)
    #print("image_path", image_path)
    escaped_path = image_path.replace(" ", "\ ")
    escaped_out_path_tokens = escaped_path.split(".")
    escaped_out_path = os.path.join(escaped_out_path_tokens[0].replace("/srcVids/", "/FRAMES/") + "/tmp.%05d.jpg")
    escaped_out_dir = os.path.dirname(escaped_out_path)
    escaped_final_path = "{}_inference.mp4".format(escaped_out_path_tokens[0].replace("srcVids", "OUT"))
    #print("escaped_path", escaped_path)
    #print("escaped_out_path", escaped_out_path)
    #print("escaped_out_dir", escaped_out_dir)
    
    !rm -rf $escaped_out_dir
    !mkdir -p $escaped_out_dir
    print("ffmpeg -y -i {0} {1}".format(escaped_path, escaped_out_path))
    !ffmpeg -y -i $escaped_path $escaped_out_path 
    
    unescaped = escaped_out_dir.replace("\ ", " ")

    sortedFiles = os.listdir(unescaped)
    sortedFiles.sort()
    
    baseOutputDirectory = None
    
    print("#"*80)
    print("  ----- Inference ----")
    print("#"*80)
    #print(sortedFiles)
    for f in sortedFiles[:300]:
        image_path = os.path.join(unescaped, f)
        #print("image_path: ", image_path)
        assert os.path.isfile(image_path)

        if not image_path.endswith("jpg"):
            continue
            
        image = skimage.io.imread(image_path)

        #Run detection
        results = model.detect([image], verbose=1)

        #Visualize results
        r = results[0]
        result = display_and_return_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    class_names, r['scores'], show_bbox=False)
    
        imgToSave = Image.fromarray(result, 'RGB')
        savePath = image_path.replace("tmp", "inference")

        imgToSave.save(savePath)
        #print("Saving: ", savePath)
        baseOutputDirectory = savePath
        #print(baseOutputDirectory)

        
    #print(baseOutputDirectory)
    
    if not baseOutputDirectory:
        continue
    
    
    print("#"*80)
    print("  ----- Encoding ----")
    print("#"*80)
    dst_tokens = baseOutputDirectory.split(".")
    savePath = "{}.%05d.jpg".format(dst_tokens[0])
    #print(dst_tokens)
    #print(savePath)
    escaped_inference_path = savePath.replace(" ", "\ ")
    escaped_inference_dir = os.path.dirname(escaped_inference_path)
    !mkdir -p $escaped_inference_dir
    #print("Final source frames: {0}".format(escaped_inference_path))
    #print("Generating final at: {0}".format(escaped_final_path))
    #print("Escaped inference path: ", escaped_inference_path)
    !ffmpeg -y -r 25 -i $escaped_inference_path -vcodec mpeg4 -crf 10 -pix_fmt yuv420p $escaped_final_path
    
    !cp $escaped_final_path /content/drive/My\ Drive/MachineLearning/trainingSets/inferenceVids/

    

/content/local_drive/srcVids
jim_gaffigan_obsessed_weddings.mp4
################################################################################
  ----- Decoding ----
################################################################################
/content/local_drive/srcVids/jim_gaffigan_obsessed_weddings.mp4
ffmpeg -y -i /content/local_drive/srcVids/jim_gaffigan_obsessed_weddings.mp4 /content/local_drive/FRAMES/jim_gaffigan_obsessed_weddings/tmp.%05d.jpg
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype -

In [33]:
%cd /content/local_drive/OUT
%ls

!cp /content/local_drive/OUT/jim_gaffigan_obsessed_weddings_inference.mp4 /content/drive/My\ Drive/MachineLearning/trainingSets/inferenceVids

/content/local_drive/OUT
jim_gaffigan_obsessed_weddings_inference.mp4


In [0]:
%cd /content/local_drive/FRAMES/jim_gaffigan_obsessed_weddings/
%ls



In [0]:

from google.colab import files
files.download('inference.00050.jpg') 

In [49]:
!ffmpeg -y -r 25 -i /content/local_drive/FRAMES/jim_gaffigan_obsessed_weddings/inference.%05d.jpg -vcodec mpeg4 -pix_fmt yuvj422p /content/local_drive/OUT/jim_gaffigan_obsessed_weddings_inference.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [50]:
%cd /content/local_drive/OUT/

files.download('jim_gaffigan_obsessed_weddings_inference.mp4') 

/content/local_drive/OUT
